## Muti Model detection

In [1]:
from ultralytics import YOLO
import cv2
import numpy as np
import math
import datetime
import os
import csv
from keras.models import load_model
from collections import deque

# Declare writer as a global variable
writer = None

# Function to perform violence detection using a TensorFlow-trained model
def violence_detection(frame):
    global writer, Q, consecutive_detection  # Access the global variables
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = cv2.resize(frame, (128, 128)).astype("float32")
    frame = frame.reshape(1, 128, 128, 3) / 255  # Reshape to match model input shape

    preds = violence_model.predict(frame)[0]
    Q.append(preds)

    results = np.array(Q).mean(axis=0)
    i = (preds > 0.50)[0]

    if i:
        consecutive_detection += 1
        if consecutive_detection >= 3:
            timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            writer.writerow(['Violence', 'True', log_filename if log_filename else 'Unknown', timestamp, preds[0]])
            consecutive_detection = 0  
    else:
        consecutive_detection = 0  

    text = "Violence: {}".format(i)
    FONT = cv2.FONT_HERSHEY_SIMPLEX 

    text_color = (0, 255, 0) 
    if i:
        text_color = (0, 0, 255)

    cv2.putText(frame, text, (35, 50), FONT, 1.25, text_color, 3)

# Initialize YOLO models
violence_model = load_model('models/modelnew.h5') 
weapon_model = YOLO('models/customWeapon.pt')
accident_model = YOLO('models/RAccidents.pt')

# Class names
weapon_class_names = ['Automatic Rifle', 'Bazooka', 'Grenade Launcher', 'Handgun', 'Knife', 'Shotgun', 'SMG', 'Sniper', 'Sword']
accident_class_names = ['fire', 'minor_accident', 'moderate_accident', 'no_accident', 'severe_accident']

# Initialize variables
consecutive_frames = 0
confidence_threshold = 0.60
required_consecutive_frames = 7
new_width = 640
new_height = 480
log_dir = "logs"

# Video file path
# video_path = "D:/KAIF/RAccident/AccidentData/val/acc1.mp4"
video_path = "D:/KAIF/RJPOLICE_HACK_587_NeuralEnvoys_3/Models/ViolenceDetection/vids/fight04.mp4"
cap = cv2.VideoCapture(video_path)

# Initialize violence detection variables
Q = deque(maxlen=128)
consecutive_detection = 0 
(W, H) = (None, None)  # Add this line to initialize W and H
current_date = datetime.datetime.now().strftime("%Y-%m-%d")
logs_folder = 'logs'
log_filename = f"{current_date}_logfile.csv"
log_path = os.path.join(logs_folder, datetime.datetime.now().strftime("%Y"), datetime.datetime.now().strftime("%b"), log_filename)

os.makedirs(os.path.dirname(log_path), exist_ok=True)  # Ensure the directory exists

with open(log_path, mode='a', newline='') as file:
    csv_writer = csv.writer(file)

    if os.path.getsize(log_path) == 0:
        csv_writer.writerow(['Detection Type', 'Class', 'Frame Number', 'Timestamp', 'Confidence'])

    writer = csv_writer  # Assign csv_writer to the global writer variable

    while True:
        (grabbed, frame) = cap.read()

        if not grabbed:
            break

        if W is None or H is None:
            (H, W) = frame.shape[:2]

        output = frame.copy()

        # Weapon detection
        weapon_results = weapon_model(frame, stream=True)

        for r in weapon_results:
            boxes = r.boxes
            for box in boxes:
                conf = box.conf[0]
                if conf >= 90.0:
                    weapon_detected = True

                    x1, y1, x2, y2 = box.xyxy[0]
                    x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
                    w, h = x2 - x1, y2 - y1
                    # cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

                    conf = math.ceil((conf * 100)) / 100

                    class_index = int(box.cls[0])
                    class_name = weapon_class_names[class_index]

                    current_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                    log_message = f"Weapon | {class_name} | {cap.get(cv2.CAP_PROP_POS_FRAMES)} | {current_time} | {conf}"

                    writer.writerow(["Weapon", class_name, cap.get(cv2.CAP_PROP_POS_FRAMES), current_time, conf])

                    # Display class name on the frame
                    # cv2.putText(frame, f"{class_name} {conf}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

                    print(log_message)

        # Violence detection
        violence_detection(frame)

        # Accident detection
        accident_results = accident_model(frame, stream=True)

        for r in accident_results:
            boxes = r.boxes
            for box in boxes:
                conf = box.conf[0]
                if conf >= confidence_threshold:
                    class_index = int(box.cls[0])
                    class_name = accident_class_names[class_index]

                    if class_name != 'no_accident':
                        accident_detected = True

                        x1, y1, x2, y2 = box.xyxy[0]
                        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
                        w, h = x2 - x1, y2 - y1

                        conf = math.ceil((conf * 100)) / 100

                        color = (0, 0, 255)  # Red for accidents
                        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)

                        current_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                        log_message = f"Accident | {class_name} | {cap.get(cv2.CAP_PROP_POS_FRAMES)} | {current_time} | {conf}"

                        writer.writerow(["Accident", class_name, cap.get(cv2.CAP_PROP_POS_FRAMES), current_time, conf])

                        # Display class name on the frame
                        cv2.putText(frame, f"{'Unusual Activity'} {conf}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

                        print(log_message)

        # Display the processed frame
        cv2.imshow("Image", frame)

        # Check for user exit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()



0: 384x640 1 Grenade Launcher, 91.8ms
Speed: 0.0ms preprocess, 91.8ms inference, 50.2ms postprocess per image at shape (1, 3, 384, 640)
1/1 [==============================] - 1s 540ms/step

0: 128x224 1 severe_accident, 99.1ms
Speed: 0.0ms preprocess, 99.1ms inference, 0.9ms postprocess per image at shape (1, 3, 128, 224)

0: 384x640 1 Grenade Launcher, 12.2ms
Speed: 2.8ms preprocess, 12.2ms inference, 6.9ms postprocess per image at shape (1, 3, 384, 640)
1/1 [==============================] - 0s 19ms/step

0: 128x224 3 severe_accidents, 7.0ms
Speed: 1.0ms preprocess, 7.0ms inference, 0.0ms postprocess per image at shape (1, 3, 128, 224)

0: 384x640 1 Grenade Launcher, 18.8ms
Speed: 0.0ms preprocess, 18.8ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)
1/1 [==============================] - 0s 19ms/step

0: 128x224 1 severe_accident, 3.9ms
Speed: 1.0ms preprocess, 3.9ms inference, 8.6ms postprocess per image at shape (1, 3, 128, 224)

0: 384x640 1 Grenade Launcher, 

In [1]:
from ultralytics import YOLO
import cv2
import numpy as np
import math
import datetime
import os
import csv
from keras.models import load_model
from collections import deque

# Declare writer as a global variable
writer = None

# Function to perform violence detection using a TensorFlow-trained model
def violence_detection(frame):
    global writer, Q, consecutive_detection  # Access the global variables
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = cv2.resize(frame, (128, 128)).astype("float32")
    frame = frame.reshape(1, 128, 128, 3) / 255  # Reshape to match model input shape

    preds = violence_model.predict(frame)[0]
    Q.append(preds)

    results = np.array(Q).mean(axis=0)
    i = (preds > 0.50)[0]

    if i:
        consecutive_detection += 1
        if consecutive_detection >= 3:
            timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            writer.writerow(['Violence', 'True', log_filename if log_filename else 'Unknown', timestamp, preds[0]])
            consecutive_detection = 0  
    else:
        consecutive_detection = 0  

    text = "Violence: {}".format(i)
    FONT = cv2.FONT_HERSHEY_SIMPLEX 

    text_color = (0, 255, 0) 
    if i:
        text_color = (0, 0, 255)

    cv2.putText(frame, text, (35, 50), FONT, 1.25, text_color, 3)

# Initialize YOLO models
violence_model = load_model('models/modelnew.h5') 
weapon_model = YOLO('models/customWeapon.pt')
accident_model = YOLO('models/RAccidents.pt')

# Class names
weapon_class_names = ['Automatic Rifle', 'Bazooka', 'Grenade Launcher', 'Handgun', 'Knife', 'Shotgun', 'SMG', 'Sniper', 'Sword']
accident_class_names = ['fire', 'minor_accident', 'moderate_accident', 'no_accident', 'severe_accident']

# Initialize variables
consecutive_frames = 0
confidence_threshold = 0.60
required_consecutive_frames = 7
new_width = 640
new_height = 480
log_dir = "logs"

# Video file path
# video_path = "D:/KAIF/RAccident/AccidentData/val/acc1.mp4"
video_path = "D:/KAIF/RJPOLICE_HACK_587_NeuralEnvoys_3/Models/ViolenceDetection/vids/fight04.mp4"
cap = cv2.VideoCapture(video_path)

if not os.path.exists('output'):
    os.mkdir('output')

# Create the VideoWriter object outside the loop
fourcc = cv2.VideoWriter_fourcc(*"MJPG")
vid_writer = cv2.VideoWriter(os.path.join('output', f"{current_date}_v_output.avi"), fourcc, 30, (W, H), True)

# Initialize violence detection variables
Q = deque(maxlen=128)
consecutive_detection = 0 
(W, H) = (None, None)  # Add this line to initialize W and H
current_date = datetime.datetime.now().strftime("%Y-%m-%d")
logs_folder = 'logs'
log_filename = f"{current_date}_logfile.csv"
log_path = os.path.join(logs_folder, datetime.datetime.now().strftime("%Y"), datetime.datetime.now().strftime("%b"), log_filename)

os.makedirs(os.path.dirname(log_path), exist_ok=True)  # Ensure the directory exists

with open(log_path, mode='a', newline='') as file:
    csv_writer = csv.writer(file)

    if os.path.getsize(log_path) == 0:
        csv_writer.writerow(['Detection Type', 'Class', 'Frame Number', 'Timestamp', 'Confidence'])

    writer = csv_writer  # Assign csv_writer to the global writer variable

    while True:
        (grabbed, frame) = cap.read()

        if not grabbed:
            break

        if W is None or H is None:
            (H, W) = frame.shape[:2]

        output = frame.copy()

        # Weapon detection
        weapon_results = weapon_model(frame, stream=True)

        for r in weapon_results:
            boxes = r.boxes
            for box in boxes:
                conf = box.conf[0]
                if conf >= 90.0:
                    weapon_detected = True

                    x1, y1, x2, y2 = box.xyxy[0]
                    x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
                    w, h = x2 - x1, y2 - y1

                    conf = math.ceil((conf * 100)) / 100

                    class_index = int(box.cls[0])
                    class_name = weapon_class_names[class_index]

                    current_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                    log_message = f"Weapon | {class_name} | {cap.get(cv2.CAP_PROP_POS_FRAMES)} | {current_time} | {conf}"

                    writer.writerow(["Weapon", class_name, cap.get(cv2.CAP_PROP_POS_FRAMES), current_time, conf])

                    print(log_message)

        # Violence detection
        violence_detection(frame)

        # Accident detection
        accident_results = accident_model(frame, stream=True)

        for r in accident_results:
            boxes = r.boxes
            for box in boxes:
                conf = box.conf[0]
                if conf >= confidence_threshold:
                    class_index = int(box.cls[0])
                    class_name = accident_class_names[class_index]

                    if class_name != 'no_accident':
                        accident_detected = True

                        x1, y1, x2, y2 = box.xyxy[0]
                        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
                        w, h = x2 - x1, y2 - y1

                        conf = math.ceil((conf * 100)) / 100

                        color = (0, 0, 255)  # Red for accidents
                        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)

                        current_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                        log_message = f"Accident | {class_name} | {cap.get(cv2.CAP_PROP_POS_FRAMES)} | {current_time} | {conf}"

                        writer.writerow(["Accident", class_name, cap.get(cv2.CAP_PROP_POS_FRAMES), current_time, conf])

                        cv2.putText(frame, f"{class_name} {conf}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

                        print(log_message)

        # Write the output frame to the video file
        vid_writer.write(output)

        # Display the processed frame
        cv2.imshow("Image", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# Release resources
cap.release()
vid_writer.release()  # Release the video writer
cv2.destroyAllWindows()


NameError: name 'current_date' is not defined

In [ ]:
from ultralytics import YOLO
import cv2
import numpy as np
import math
import datetime
import os
import csv
from keras.models import load_model
from collections import deque

# Declare writer as a global variable
writer = None

# Function to perform violence detection using a TensorFlow-trained model
def violence_detection(frame):
    global writer, Q, consecutive_detection  # Access the global variables
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = cv2.resize(frame, (128, 128)).astype("float32")
    frame = frame.reshape(1, 128, 128, 3) / 255  # Reshape to match model input shape

    preds = violence_model.predict(frame)[0]
    Q.append(preds)

    results = np.array(Q).mean(axis=0)
    i = (preds > 0.50)[0]

    if i:
        consecutive_detection += 1
        if consecutive_detection >= 3:
            timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            writer.writerow(['Violence', 'True', log_filename if log_filename else 'Unknown', timestamp, preds[0]])
            consecutive_detection = 0  
    else:
        consecutive_detection = 0  

    text = "Violence: {}".format(i)
    FONT = cv2.FONT_HERSHEY_SIMPLEX 

    text_color = (0, 255, 0) 
    if i:
        text_color = (0, 0, 255)

    cv2.putText(frame, text, (35, 50), FONT, 1.25, text_color, 3)

# Initialize YOLO models
violence_model = load_model('models/modelnew.h5') 
weapon_model = YOLO('models/customWeapon.pt')
accident_model = YOLO('models/RAccidents.pt')

# Class names
weapon_class_names = ['Automatic Rifle', 'Bazooka', 'Grenade Launcher', 'Handgun', 'Knife', 'Shotgun', 'SMG', 'Sniper', 'Sword']
accident_class_names = ['fire', 'minor_accident', 'moderate_accident', 'no_accident', 'severe_accident']

# Initialize variables
consecutive_frames = 0
confidence_threshold = 0.60
required_consecutive_frames = 7
new_width = 640
new_height = 480
log_dir = "logs"

# Video file path
video_path = "D:/KAIF/RAccident/AccidentData/val/acc1.mp4"
# video_path = "D:/KAIF/RJPOLICE_HACK_587_NeuralEnvoys_3/Models/ViolenceDetection/vids/fight04.mp4"
cap = cv2.VideoCapture(video_path)

if not os.path.exists('output'):
    os.mkdir('output')


# Initialize violence detection variables
Q = deque(maxlen=128)
consecutive_detection = 0 
(W, H) = (None, None)  # Add this line to initialize W and H
current_date = datetime.datetime.now().strftime("%Y-%m-%d")
logs_folder = 'logs'
log_filename = f"{current_date}_logfile.csv"
log_path = os.path.join(logs_folder, datetime.datetime.now().strftime("%Y"), datetime.datetime.now().strftime("%b"), log_filename)

os.makedirs(os.path.dirname(log_path), exist_ok=True)  # Ensure the directory exists

# Create the VideoWriter object outside the loop
fourcc = cv2.VideoWriter_fourcc(*"MJPG")
vid_writer = cv2.VideoWriter(os.path.join('output', f"{current_date}_v_output.avi"), fourcc, 30, (W, H), True)

with open(log_path, mode='a', newline='') as file:
    csv_writer = csv.writer(file)

    if os.path.getsize(log_path) == 0:
        csv_writer.writerow(['Detection Type', 'Class', 'Frame Number', 'Timestamp', 'Confidence'])

    writer = csv_writer  # Assign csv_writer to the global writer variable

    while True:
        (grabbed, frame) = cap.read()

        if not grabbed:
            break

        if W is None or H is None:
            (H, W) = frame.shape[:2]

        output = frame.copy()

        # Weapon detection
        weapon_results = weapon_model(frame, stream=True)
        for r in weapon_results:
            # Process bounding boxes and labels for weapons
            boxes = r.boxes
            for box in boxes:
                conf = box.conf[0]
                if conf >= 90.0:
                    weapon_detected = True

                    x1, y1, x2, y2 = box.xyxy[0]
                    x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
                    w, h = x2 - x1, y2 - y1

                    conf = math.ceil((conf * 100)) / 100

                    class_index = int(box.cls[0])
                    class_name = weapon_class_names[class_index]

                    current_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                    log_message = f"Weapon | {class_name} | {cap.get(cv2.CAP_PROP_POS_FRAMES)} | {current_time} | {conf}"

                    writer.writerow(["Weapon", class_name, cap.get(cv2.CAP_PROP_POS_FRAMES), current_time, conf])

                    # Draw bounding box and label on the output frame
                    cv2.rectangle(output, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    cv2.putText(output, f"{class_name} {conf}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

                    print(log_message)

        # Violence detection
        violence_detection(output)

        # Accident detection
        accident_results = accident_model(frame, stream=True)
        for r in accident_results:
            # Process bounding boxes and labels for accidents
            boxes = r.boxes
            for box in boxes:
                conf = box.conf[0]
                if conf >= confidence_threshold:
                    class_index = int(box.cls[0])
                    class_name = accident_class_names[class_index]

                    if class_name != 'no_accident':
                        accident_detected = True

                        x1, y1, x2, y2 = box.xyxy[0]
                        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
                        w, h = x2 - x1, y2 - y1

                        conf = math.ceil((conf * 100)) / 100

                        color = (0, 0, 255)  # Red for accidents
                        cv2.rectangle(output, (x1, y1), (x2, y2), color, 2)

                        current_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                        log_message = f"Accident | {class_name} | {cap.get(cv2.CAP_PROP_POS_FRAMES)} | {current_time} | {conf}"

                        writer.writerow(["Accident", class_name, cap.get(cv2.CAP_PROP_POS_FRAMES), current_time, conf])

                        # Draw bounding box and label on the output frame
                        cv2.putText(output, f"{'Unusaual Activity'} {conf}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

                        print(log_message)

        # Draw bounding boxes and labels for violence detection on the output frame
        cv2.imshow("Image", output)

        # Write the output frame to the video file
        vid_writer.write(output)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# Release resources
cap.release()
vid_writer.release()  # Release the video writer
cv2.destroyAllWindows()

## working

In [3]:
from ultralytics import YOLO
import cv2
import numpy as np
import math
import datetime
import os
import csv
from keras.models import load_model
from collections import deque

# Declare writer as a global variable
writer = None

# Function to perform violence detection using a TensorFlow-trained model
def violence_detection(frame):
    global writer, Q, consecutive_detection  # Access the global variables
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = cv2.resize(frame, (128, 128)).astype("float32")
    frame = frame.reshape(1, 128, 128, 3) / 255  # Reshape to match model input shape

    preds = violence_model.predict(frame)[0]
    Q.append(preds)

    results = np.array(Q).mean(axis=0)
    i = (preds > 0.50)[0]

    if i:
        consecutive_detection += 1
        if consecutive_detection >= 3:
            timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            writer.writerow(['Violence', 'True', filename , timestamp, preds[0]])
            consecutive_detection = 0  
    else:
        consecutive_detection = 0  

    text = "Violence: {}".format(i)
    FONT = cv2.FONT_HERSHEY_SIMPLEX 

    text_color = (0, 255, 0) 
    if i:
        text_color = (0, 0, 255)

    cv2.putText(frame, text, (35, 50), FONT, 1.25, text_color, 3)

# Initialize YOLO models
violence_model = load_model('models/modelnew.h5') 
weapon_model = YOLO('models/customWeapon.pt')
accident_model = YOLO('models/RAccidents.pt')

# Class names
weapon_class_names = ['Automatic Rifle', 'Bazooka', 'Grenade Launcher', 'Handgun', 'Knife', 'Shotgun', 'SMG', 'Sniper', 'Sword']
accident_class_names = ['fire', 'minor_accident', 'moderate_accident', 'no_accident', 'severe_accident']

# Initialize variables
consecutive_frames = 0
confidence_threshold = 0.60
required_consecutive_frames = 7
new_width = 640
new_height = 480
log_dir = "logs"

# Video file path
video_path = "D:/KAIF/combined/vids/cam2.mp4"
# video_path = "D:/KAIF/RJPOLICE_HACK_587_NeuralEnvoys_3/Models/ViolenceDetection/vids/fight04.mp4"
filename = os.path.basename(video_path)
cap = cv2.VideoCapture(video_path)

if not os.path.exists('output'):
    os.mkdir('output')

# Create the VideoWriter object outside the loop
fourcc = cv2.VideoWriter_fourcc(*"MJPG")
vid_writer = cv2.VideoWriter(os.path.join('output', f"{current_date}_v_output.avi"), fourcc, 30, (W, H), True)

# Initialize violence detection variables
Q = deque(maxlen=128)
consecutive_detection = 0 
(W, H) = (None, None)  # Add this line to initialize W and H
current_date = datetime.datetime.now().strftime("%Y-%m-%d")
logs_folder = 'logs'
log_filename = f"{current_date}_logfile.csv"
log_path = os.path.join(logs_folder, datetime.datetime.now().strftime("%Y"), datetime.datetime.now().strftime("%b"), log_filename)

os.makedirs(os.path.dirname(log_path), exist_ok=True)  # Ensure the directory exists

with open(log_path, mode='a', newline='') as file:
    csv_writer = csv.writer(file)

    if os.path.getsize(log_path) == 0:
        csv_writer.writerow(['Detection Type', 'Class', 'FileName/cam', 'Timestamp', 'Confidence'])

    writer = csv_writer  # Assign csv_writer to the global writer variable

    while True:
        (grabbed, frame) = cap.read()

        if not grabbed:
            break

        if W is None or H is None:
            (H, W) = frame.shape[:2]

        output = frame.copy()

        # Weapon detection
        weapon_results = weapon_model(frame, stream=True)
        for r in weapon_results:
            # Process bounding boxes and labels for weapons
            boxes = r.boxes
            for box in boxes:
                conf = box.conf[0]
                if conf >= 90.0:
                    weapon_detected = True

                    x1, y1, x2, y2 = box.xyxy[0]
                    x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
                    w, h = x2 - x1, y2 - y1

                    conf = math.ceil((conf * 100)) / 100

                    class_index = int(box.cls[0])
                    class_name = weapon_class_names[class_index]

                    current_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                    log_message = f"Weapon | {class_name} | {cap.get(cv2.CAP_PROP_POS_FRAMES)} | {current_time} | {conf}"

                    writer.writerow(["Weapon", class_name, filename, current_time, conf])

                    # Draw bounding box and label on the output frame
                    cv2.rectangle(output, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    cv2.putText(output, f"{'Weapon'} {conf}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

                    print(log_message)

        # Violence detection
        violence_detection(output)

        # Accident detection
        accident_results = accident_model(frame, stream=True)
        for r in accident_results:
            # Process bounding boxes and labels for accidents
            boxes = r.boxes
            for box in boxes:
                conf = box.conf[0]
                if conf >= confidence_threshold:
                    class_index = int(box.cls[0])
                    class_name = accident_class_names[class_index]

                    if class_name != 'no_accident':
                        accident_detected = True

                        x1, y1, x2, y2 = box.xyxy[0]
                        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
                        w, h = x2 - x1, y2 - y1

                        conf = math.ceil((conf * 100)) / 100

                        color = (0, 0, 255)  # Red for accidents
                        cv2.rectangle(output, (x1, y1), (x2, y2), color, 2)

                        current_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                        log_message = f"Accident | {class_name} | {cap.get(cv2.CAP_PROP_POS_FRAMES)} | {current_time} | {conf}"

                        writer.writerow(["Accident", class_name, filename, current_time, conf])

                        # Draw bounding box and label on the output frame
                        cv2.putText(output, f"{'Unusual Activity'} {conf}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

                        print(log_message)

        # Draw bounding boxes and labels for violence detection on the output frame
        cv2.imshow("Image", output)

        # Write the output frame to the video file
        vid_writer.write(output)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# Release resources
cap.release()
vid_writer.release()  # Release the video writer
cv2.destroyAllWindows()

In [ ]:
from ultralytics import YOLO
import cv2
import numpy as np
import math
import datetime
import os
import csv
from keras.models import load_model
from collections import deque

# Declare writer as a global variable
writer = None
i = False
FONT = cv2.FONT_HERSHEY_SIMPLEX
text_color = (0, 0, 255)

# Function to perform violence detection using a TensorFlow-trained model
def violence_detection(frame):
    global writer, Q, consecutive_detection, i  # Access the global variables
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = cv2.resize(frame, (128, 128)).astype("float32")
    frame = frame.reshape(1, 128, 128, 3) / 255  # Reshape to match model input shape

    preds = violence_model.predict(frame)[0]
    Q.append(preds)

    results = np.array(Q).mean(axis=0)
    i = (preds > 0.50)[0]

    if i:
        consecutive_detection += 1
        if consecutive_detection >= 3:
            timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            writer.writerow(['Violence', 'True', log_filename if log_filename else 'Unknown', timestamp, preds[0]])
            consecutive_detection = 0
    else:
        consecutive_detection = 0

    text = "Violence: {}".format(i)
    FONT = cv2.FONT_HERSHEY_SIMPLEX

    text_color = (0, 255, 0)
    if i:
        text_color = (0, 0, 255)

    cv2.putText(frame, text, (35, 50), FONT, 1.25, text_color, 3)

# Initialize YOLO models
violence_model = load_model('models/modelnew.h5')
weapon_model = YOLO('models/customWeapon.pt')
accident_model = YOLO('models/RAccidents.pt')

# Class names
weapon_class_names = ['Automatic Rifle', 'Bazooka', 'Grenade Launcher', 'Handgun', 'Knife', 'Shotgun', 'SMG', 'Sniper', 'Sword']
accident_class_names = ['fire', 'minor_accident', 'moderate_accident', 'no_accident', 'severe_accident']

# Initialize variables
consecutive_frames = 0
confidence_threshold = 0.60
required_consecutive_frames = 7
new_width = 640
new_height = 480
log_dir = "logs"

# Video file path
video_path = "D:/KAIF/RAccident/AccidentData/val/acc1.mp4"
cap = cv2.VideoCapture(video_path)

# Initialize violence detection variables
Q = deque(maxlen=128)
consecutive_detection = 0
(W, H) = (None, None)  # Add this line to initialize W and H
current_date = datetime.datetime.now().strftime("%Y-%m-%d")
logs_folder = 'logs'
log_filename = f"{current_date}_logfile.csv"
log_path = os.path.join(logs_folder, datetime.datetime.now().strftime("%Y"), datetime.datetime.now().strftime("%b"), log_filename)

os.makedirs(os.path.dirname(log_path), exist_ok=True)  # Ensure the directory exists

with open(log_path, mode='a', newline='') as file:
    csv_writer = csv.writer(file)

    if os.path.getsize(log_path) == 0:
        csv_writer.writerow(['Detection Type', 'Class', 'Frame Number', 'Timestamp', 'Confidence'])

    writer = csv_writer  # Assign csv_writer to the global writer variable

    # Create the "outputs" folder if it doesn't exist
    output_folder = 'outputs'
    os.makedirs(output_folder, exist_ok=True)

    # Output video file path
    output_video_path = os.path.join(output_folder, 'output_video.avi')

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")  # You can choose a different codec if needed
    output_video = cv2.VideoWriter(output_video_path, fourcc, 20.0, (new_width, new_height))

    while True:
        (grabbed, frame) = cap.read()

        if not grabbed:
            break

        if W is None or H is None:
            (H, W) = frame.shape[:2]

        output = frame.copy()

        # Weapon detection
        weapon_results = weapon_model(frame, stream=True)

        for r in weapon_results:
            boxes = r.boxes
            for box in boxes:
                conf = box.conf[0]
                if conf >= 90.0:
                    weapon_detected = True

                    x1, y1, x2, y2 = box.xyxy[0]
                    x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
                    w, h = x2 - x1, y2 - y1

                    conf = math.ceil((conf * 100)) / 100

                    class_index = int(box.cls[0])
                    class_name = weapon_class_names[class_index]

                    current_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                    log_message = f"Weapon | {class_name} | {cap.get(cv2.CAP_PROP_POS_FRAMES)} | {current_time} | {conf}"

                    writer.writerow(["Weapon", class_name, cap.get(cv2.CAP_PROP_POS_FRAMES), current_time, conf])

                    # Display class name on the frame
                    # cv2.putText(frame, f"{'weapon'} {conf}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 255), 2)
                    cv2.putText(frame, f"{'Unusual Activity'} {conf}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)


                    print(log_message)

        # Violence detection
        violence_detection(frame)

        # Accident detection
        accident_results = accident_model(frame, stream=True)

        for r in accident_results:
            boxes = r.boxes
            for box in boxes:
                conf = box.conf[0]
                if conf >= confidence_threshold:
                    class_index = int(box.cls[0])
                    class_name = accident_class_names[class_index]

                    if class_name != 'no_accident':
                        accident_detected = True

                        x1, y1, x2, y2 = box.xyxy[0]
                        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
                        w, h = x2 - x1, y2 - y1

                        conf = math.ceil((conf * 100)) / 100

                        color = (0, 0, 255)  # Red for accidents
                        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)

                        current_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                        log_message = f"Accident | {class_name} | {cap.get(cv2.CAP_PROP_POS_FRAMES)} | {current_time} | {conf}"

                        writer.writerow(["Accident", class_name, cap.get(cv2.CAP_PROP_POS_FRAMES), current_time, conf])

                        # Display class name on the frame
                        cv2.putText(frame, f"{'Unusual Activity'} {conf}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

                        print(log_message)

        # Display the processed frame
        cv2.imshow("Image", frame)

        # Write the frame to the output video file
        output_video.write(frame)

        # Display violence prediction on the frame
        if i:
            violence_text = "Violence: True"
        else:
            violence_text = "Violence: False"

        cv2.putText(frame, violence_text, (35, 80), FONT, 1.25, text_color, 3)

        # Check for user exit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the VideoWriter object
    output_video.release()

cap.release()
cv2.destroyAllWindows()


## Multiple videos

In [ ]:
from ultralytics import YOLO
import cv2
import numpy as np
import math
import datetime
import os
import csv
from keras.models import load_model
from collections import deque


def violence_detection(frame, writer, Q, consecutive_detection, violence_model):
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = cv2.resize(frame, (128, 128)).astype("float32")
    frame = frame.reshape(1, 128, 128, 3) / 255  # Reshape to match model input shape

    preds = violence_model.predict(frame)[0]
    Q.append(preds)

    results = np.array(Q).mean(axis=0)
    i = (preds > 0.50)[0]

    if i:
        consecutive_detection.value += 1
        if consecutive_detection.value >= 3:
            timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            writer.writerow(['Violence', 'True', log_filename if log_filename else 'Unknown', timestamp, preds[0]])
            consecutive_detection.value = 0
    else:
        consecutive_detection.value = 0

    text = "Violence: {}".format(i)
    FONT = cv2.FONT_HERSHEY_SIMPLEX

    text_color = (0, 255, 0)
    if i:
        text_color = (0, 0, 255)

    cv2.putText(frame, text, (35, 50), FONT, 1.25, text_color, 3)

def process_video(video_path, writer, Q, consecutive_detection, violence_model, weapon_model, accident_model):
    cap = cv2.VideoCapture(video_path)
    W, H = None, None

    while True:
        grabbed, frame = cap.read()

        if not grabbed:
            break

        if W is None or H is None:
            (H, W) = frame.shape[:2]

        output = frame.copy()

        # Weapon detection
        weapon_results = weapon_model(frame, stream=True)

        for r in weapon_results:
            boxes = r.boxes
            for box in boxes:
                conf = box.conf[0]
                if conf >= 90.0:
                    weapon_detected = True

                    x1, y1, x2, y2 = box.xyxy[0]
                    x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
                    w, h = x2 - x1, y2 - y1
                    # cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

                    conf = math.ceil((conf * 100)) / 100

                    class_index = int(box.cls[0])
                    class_name = weapon_class_names[class_index]

                    current_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                    log_message = f"Weapon | {class_name} | {cap.get(cv2.CAP_PROP_POS_FRAMES)} | {current_time} | {conf}"

                    writer.writerow(["Weapon", class_name, cap.get(cv2.CAP_PROP_POS_FRAMES), current_time, conf])

                    # Display class name on the frame
                    # cv2.putText(frame, f"{class_name} {conf}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

                    print(log_message)

        # Violence detection
        violence_detection(frame, writer, Q, consecutive_detection, violence_model)

        # Accident detection
        accident_results = accident_model(frame, stream=True)

        for r in accident_results:
            boxes = r.boxes
            for box in boxes:
                conf = box.conf[0]
                if conf >= confidence_threshold:
                    class_index = int(box.cls[0])
                    class_name = accident_class_names[class_index]

                    if class_name != 'no_accident':
                        accident_detected = True

                        x1, y1, x2, y2 = box.xyxy[0]
                        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
                        w, h = x2 - x1, y2 - y1

                        conf = math.ceil((conf * 100)) / 100

                        color = (0, 0, 255)  # Red for accidents
                        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)

                        current_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                        log_message = f"Accident | {class_name} | {cap.get(cv2.CAP_PROP_POS_FRAMES)} | {current_time} | {conf}"

                        writer.writerow(["Accident", class_name, cap.get(cv2.CAP_PROP_POS_FRAMES), current_time, conf])

                        # Display class name on the frame
                        cv2.putText(frame, f"{class_name} {conf}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

                        print(log_message)

        # Display the processed frame
        cv2.imshow("Image", frame)

        # Check for user exit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == '__main__':
    # Initialize YOLO models
    violence_model = load_model('models/modelnew.h5')  # Replace with the actual path
    weapon_model = YOLO('models/customWeapon.pt')
    accident_model = YOLO('models/RAccidents.pt')

    # Class names
    weapon_class_names = ['Automatic Rifle', 'Bazooka', 'Grenade Launcher', 'Handgun', 'Knife', 'Shotgun', 'SMG', 'Sniper', 'Sword']
    accident_class_names = ['fire', 'minor_accident', 'moderate_accident', 'no_accident', 'severe_accident']

    # Initialize variables
    consecutive_frames = 0
    confidence_threshold = 0.60
    required_consecutive_frames = 7
    new_width = 640
    new_height = 480
    log_dir = "logs"

    # Video file path
    video_path = "D:/KAIF/RJPOLICE_HACK_587_NeuralEnvoys_3/Models/ViolenceDetection/vids/fight04.mp4"
    cap = cv2.VideoCapture(video_path)

    # Initialize violence detection variables
    Q = deque(maxlen=128)
    consecutive_detection = Manager().Value('i', 0)
    (W, H) = (None, None)

    current_date = datetime.datetime.now().strftime("%Y-%m-%d")
    logs_folder = 'logs'
    log_filename = f"{current_date}_logfile.csv"
    log_path = os.path.join(logs_folder, datetime.datetime.now().strftime("%Y"), datetime.datetime.now().strftime("%b"), log_filename)

    os.makedirs(os.path.dirname(log_path), exist_ok=True)  # Ensure the directory exists

    with open(log_path, mode='a', newline='') as file:
        csv_writer = csv.writer(file)

        if os.path.getsize(log_path) == 0:
            csv_writer.writerow(['Detection Type', 'Class', 'Frame Number', 'Timestamp', 'Confidence'])

        writer = csv_writer  # Assign csv_writer to the global writer variable

        # Call the process_video function
        process_video(video_path, writer, Q, consecutive_detection, violence_model, weapon_model, accident_model)


In [4]:
from ultralytics import YOLO
import cv2
import numpy as np
import math
import datetime
import os
import csv
from keras.models import load_model
from collections import deque
from multiprocessing import Manager

def violence_detection(frame, writer, Q, consecutive_detection, violence_model):
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = cv2.resize(frame, (128, 128)).astype("float32")
    frame = frame.reshape(1, 128, 128, 3) / 255  # Reshape to match model input shape

    preds = violence_model.predict(frame)[0]
    Q.append(preds)

    results = np.array(Q).mean(axis=0)
    i = (preds > 0.50)[0]

    if i:
        consecutive_detection.value += 1
        if consecutive_detection.value >= 3:
            timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            writer.writerow(['Violence', 'True', log_filename if log_filename else 'Unknown', timestamp, preds[0]])
            consecutive_detection.value = 0
    else:
        consecutive_detection.value = 0

    text = "Violence: {}".format(i)
    FONT = cv2.FONT_HERSHEY_SIMPLEX

    text_color = (0, 255, 0)
    if i:
        text_color = (0, 0, 255)

    cv2.putText(frame, text, (35, 50), FONT, 1.25, text_color, 3)

def process_video(video_path, writer, Q, consecutive_detection, violence_model, weapon_model, accident_model):
    cap = cv2.VideoCapture(video_path)
    W, H = None, None

    while True:
        grabbed, frame = cap.read()

        if not grabbed:
            break

        if W is None or H is None:
            (H, W) = frame.shape[:2]

        output = frame.copy()

        # Weapon detection
        weapon_results = weapon_model(frame, stream=True)

        for r in weapon_results:
            boxes = r.boxes
            for box in boxes:
                conf = box.conf[0]
                if conf >= 90.0:
                    weapon_detected = True

                    x1, y1, x2, y2 = box.xyxy[0]
                    x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
                    w, h = x2 - x1, y2 - y1

                    conf = math.ceil((conf * 100)) / 100

                    class_index = int(box.cls[0])
                    class_name = weapon_class_names[class_index]

                    current_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                    log_message = f"Weapon | {class_name} | {cap.get(cv2.CAP_PROP_POS_FRAMES)} | {current_time} | {conf}"

                    writer.writerow(["Weapon", class_name, cap.get(cv2.CAP_PROP_POS_FRAMES), current_time, conf])

                    print(log_message)

        # Violence detection
        violence_detection(frame, writer, Q, consecutive_detection, violence_model)

        # Accident detection
        accident_results = accident_model(frame, stream=True)

        for r in accident_results:
            boxes = r.boxes
            for box in boxes:
                conf = box.conf[0]
                if conf >= confidence_threshold:
                    class_index = int(box.cls[0])
                    class_name = accident_class_names[class_index]

                    if class_name != 'no_accident':
                        accident_detected = True

                        x1, y1, x2, y2 = box.xyxy[0]
                        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
                        w, h = x2 - x1, y2 - y1

                        conf = math.ceil((conf * 100)) / 100

                        color = (0, 0, 255)  # Red for accidents
                        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)

                        current_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                        log_message = f"Accident | {class_name} | {cap.get(cv2.CAP_PROP_POS_FRAMES)} | {current_time} | {conf}"

                        writer.writerow(["Accident", class_name, cap.get(cv2.CAP_PROP_POS_FRAMES), current_time, conf])

                        print(log_message)
        # Display the processed frame
        cv2.imshow(video_path, frame)

        # Check for user exit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == '__main__':
    # Initialize YOLO models
    violence_model = load_model('models/modelnew.h5')  # Replace with the actual path
    weapon_model = YOLO('models/customWeapon.pt')
    accident_model = YOLO('models/RAccidents.pt')

    # Class names
    weapon_class_names = ['Automatic Rifle', 'Bazooka', 'Grenade Launcher', 'Handgun', 'Knife', 'Shotgun', 'SMG', 'Sniper', 'Sword']
    accident_class_names = ['fire', 'minor_accident', 'moderate_accident', 'no_accident', 'severe_accident']

    # Initialize variables
    consecutive_frames = 0
    confidence_threshold = 0.60
    required_consecutive_frames = 7
    new_width = 640
    new_height = 480
    log_dir = "logs"

    # Video file paths
    video_files = [
        "D:/KAIF/RJPOLICE_HACK_587_NeuralEnvoys_3/Models/ViolenceDetection/vids/fight04.mp4",
        "D:/KAIF/RJPOLICE_HACK_587_NeuralEnvoys_3/Models/ViolenceDetection/vids/fight04.mp4",
        # Add more video file paths as needed
    ]

    for video_path in video_files:
        # Initialize violence detection variables for each video
        Q = deque(maxlen=128)
        consecutive_detection = Manager().Value('i', 0)
        (W, H) = (None, None)

        current_date = datetime.datetime.now().strftime("%Y-%m-%d")
        logs_folder = 'logs'
        log_filename = f"{current_date}_logfile.csv"
        log_path = os.path.join(logs_folder, datetime.datetime.now().strftime("%Y"), datetime.datetime.now().strftime("%b"), log_filename)

        os.makedirs(os.path.dirname(log_path), exist_ok=True)  # Ensure the directory exists

        with open(log_path, mode='a', newline='') as file:
            csv_writer = csv.writer(file)

            if os.path.getsize(log_path) == 0:
                csv_writer.writerow(['Detection Type', 'Class', 'Frame Number', 'Timestamp', 'Confidence'])

            writer = csv_writer  # Assign csv_writer to the global writer variable

            # Call the process_video function for each video
            process_video(video_path, writer, Q, consecutive_detection, violence_model, weapon_model, accident_model)

            # Display the final frame for the video before moving to the next one
            cv2.waitKey(0)



0: 384x640 1 Grenade Launcher, 69.9ms
Speed: 2.0ms preprocess, 69.9ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
1/1 [==============================] - 1s 761ms/step

0: 128x224 1 severe_accident, 118.7ms
Speed: 1.0ms preprocess, 118.7ms inference, 0.0ms postprocess per image at shape (1, 3, 128, 224)

0: 384x640 1 Grenade Launcher, 36.8ms
Speed: 0.4ms preprocess, 36.8ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
1/1 [==============================] - 0s 33ms/step

0: 128x224 3 severe_accidents, 15.2ms
Speed: 1.0ms preprocess, 15.2ms inference, 2.2ms postprocess per image at shape (1, 3, 128, 224)

0: 384x640 1 Grenade Launcher, 31.9ms
Speed: 8.4ms preprocess, 31.9ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)
1/1 [==============================] - 0s 33ms/step

0: 128x224 1 severe_accident, 14.1ms
Speed: 1.0ms preprocess, 14.1ms inference, 4.6ms postprocess per image at shape (1, 3, 128, 224)

0: 384x640 1 Grenade Launc

In [ ]:
from ultralytics import YOLO
import cv2
import numpy as np
import math
import datetime
import os
import csv
from keras.models import load_model
from collections import deque
from multiprocessing import Process

# Function to perform violence detection using a TensorFlow-trained model
def violence_detection(frame, violence_model, writer, Q, consecutive_detection):
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = cv2.resize(frame, (128, 128)).astype("float32")
    frame = frame.reshape(1, 128, 128, 3) / 255  # Reshape to match model input shape

    preds = violence_model.predict(frame)[0]
    Q.append(preds)

    results = np.array(Q).mean(axis=0)
    i = (preds > 0.50)[0]

    if i:
        consecutive_detection += 1
        if consecutive_detection >= 3:
            timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            writer.writerow(['Violence', 'True', log_filename if log_filename else 'Unknown', timestamp, preds[0]])
            consecutive_detection = 0  
    else:
        consecutive_detection = 0  

    text = "Violence: {}".format(i)
    FONT = cv2.FONT_HERSHEY_SIMPLEX 

    text_color = (0, 255, 0) 
    if i:
        text_color = (0, 0, 255)

    cv2.putText(frame, text, (35, 50), FONT, 1.25, text_color, 3)

# Function to process a video file
def process_video(video_path, weapon_model, accident_model, violence_model, writer):
    cap = cv2.VideoCapture(video_path)

    # Initialize violence detection variables
    Q = deque(maxlen=128)
    consecutive_detection = 0 
    (W, H) = (None, None)

    while True:
        (grabbed, frame) = cap.read()

        if not grabbed:
            break

        if W is None or H is None:
            (H, W) = frame.shape[:2]

        output = frame.copy()

        # Weapon detection
        weapon_results = weapon_model(frame, stream=True)

        for r in weapon_results:
            boxes = r.boxes
            for box in boxes:
                conf = box.conf[0]
                if conf >= 90.0:
                    weapon_detected = True

                    x1, y1, x2, y2 = box.xyxy[0]
                    x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
                    w, h = x2 - x1, y2 - y1
                    # cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

                    conf = math.ceil((conf * 100)) / 100

                    class_index = int(box.cls[0])
                    class_name = weapon_class_names[class_index]

                    current_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                    log_message = f"Weapon | {class_name} | {cap.get(cv2.CAP_PROP_POS_FRAMES)} | {current_time} | {conf}"

                    writer.writerow(["Weapon", class_name, cap.get(cv2.CAP_PROP_POS_FRAMES), current_time, conf])

                    # Display class name on the frame
                    # cv2.putText(frame, f"{class_name} {conf}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

                    print(log_message)

        # Violence detection
        violence_detection(frame, violence_model, writer, Q, consecutive_detection)

        # Accident detection
        accident_results = accident_model(frame, stream=True)

        for r in accident_results:
            boxes = r.boxes
            for box in boxes:
                conf = box.conf[0]
                if conf >= confidence_threshold:
                    class_index = int(box.cls[0])
                    class_name = accident_class_names[class_index]

                    if class_name != 'no_accident':
                        accident_detected = True

                        x1, y1, x2, y2 = box.xyxy[0]
                        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
                        w, h = x2 - x1, y2 - y1

                        conf = math.ceil((conf * 100)) / 100

                        color = (0, 0, 255)  # Red for accidents
                        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)

                        current_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                        log_message = f"Accident | {class_name} | {cap.get(cv2.CAP_PROP_POS_FRAMES)} | {current_time} | {conf}"

                        writer.writerow(["Accident", class_name, cap.get(cv2.CAP_PROP_POS_FRAMES), current_time, conf])

                        # Display class name on the frame
                        cv2.putText(frame, f"{class_name} {conf}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

                        print(log_message)


        # Display the processed frame
        cv2.imshow("Image", frame)

        # Check for user exit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    # Initialize YOLO models
    weapon_model = YOLO('models/RAccidents.pt')
    accident_model = YOLO('models/RAccidents.pt')

    # Load violence detection model
    violence_model = load_model('models/modelnew.h5')

    # Video paths
    video_paths = [
        "D:/KAIF/RJPOLICE_HACK_587_NeuralEnvoys_3/Models/ViolenceDetection/vids/fight04.mp4",
        "D:/KAIF/RJPOLICE_HACK_587_NeuralEnvoys_3/Models/ViolenceDetection/vids/fight05.mp4",
    ]

    processes = []

    for video_path in video_paths:
        log_filename = f"{datetime.datetime.now().strftime('%Y-%m-%d')}_logfile.csv"
        with open(log_filename, mode='a', newline='') as file:
            csv_writer = csv.writer(file)

            if os.path.getsize(log_filename) == 0:
                csv_writer.writerow(['Detection Type', 'Class', 'Frame Number', 'Timestamp', 'Confidence'])

            writer = csv_writer  # Assign csv_writer to the global writer variable

            process = Process(target=process_video, args=(video_path, weapon_model, accident_model, violence_model, writer))
            processes.append(process)
            process.start()

    for process in processes:
        process.join()
